# Load data

In [4]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv("/content/train_features.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/train_features.csv'

In [ ]:
labels = pd.read_csv("/content/train_labels.csv")

In [ ]:
print(data.isnull().sum())

In [ ]:
data

In [ ]:
num_patient = np.unique(data.values[:,0]).shape[0]

In [ ]:
data_patients = data.values
data_patients = data_patients

In [ ]:
patients_imputed = np.zeros((np.unique(data_patients[:,0]).shape[0], 35))

## Preprocessing for Task A, B

In [ ]:
for idx_patient in range(np.unique(data_patients[:,0]).shape[0]):
  patient = data_patients[idx_patient * 12 : idx_patient * 12 + 12, :]
  print(patient)
  print(patient.shape)

  patient = np.nan_to_num(np.nanmean(patient, axis=0), nan=0)[2:]
  print(patient)
  print(patient.shape)

  patients_imputed[idx_patient] = patient

In [ ]:
patients_imputed

In [ ]:
labels = labels.values

In [ ]:
labels

In [ ]:
labels = labels[:,1:12]

In [ ]:
labels.shape

In [ ]:
patients_imputed.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization

In [ ]:
from keras.callbacks import ModelCheckpoint

In [ ]:
#create callback
filepath = 'binary_classification.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=True,
                             mode='min')

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=4, min_lr=0.00001)

callbacks = [checkpoint, reduce_lr]

In [ ]:
model = Sequential()
model.add(Dense(1024, input_dim=patients_imputed.shape[1], kernel_initializer='he_uniform', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512, kernel_initializer='he_uniform', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512, kernel_initializer='he_uniform', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(labels.shape[1], activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')

In [ ]:
labels.shape

In [ ]:
model.fit(patients_imputed, labels, epochs=50, validation_split=0.2,
                  callbacks=callbacks)

In [ ]:
filepath

In [ ]:
model = load_model(filepath)

In [ ]:
predicted_patients = model.predict(patients_imputed[15000:])

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(labels[15000:], predicted_patients)

In [ ]:
labels

In [ ]:
predicted_patients[0]

In [ ]:
df_test_features  = pd.read_csv('test_features.csv', index_col=0)

In [ ]:
IdF_test_ = df_test_features.index

In [ ]:
IdF_test_unique = pd.unique(IdF_test_)

In [ ]:
IdF_test_unique

In [ ]:
Labels=['pid','LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total',
         'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
         'LABEL_Bilirubin_direct', 'LABEL_EtCO2','LABEL_Sepsis','LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']
output = pd.DataFrame(np.array([[0.0]*16]), columns=Labels)

In [ ]:
df_test_features

In [ ]:
test_data_patient = df_test_features.values

In [ ]:
test_data_patient = test_data_patient[:,1:]

In [ ]:
test_data_patient.shape

In [ ]:
151968//12

In [ ]:
len(IdF_test_unique)

In [ ]:
test_data_patient

In [ ]:
test_patients_imputed = np.zeros((len(IdF_test_unique), 35))

In [ ]:
for idx_patient in range(len(IdF_test_unique)):
  patient = test_data_patient[idx_patient * 12 : idx_patient * 12 + 12, :]
  print(patient)
  print(patient.shape)

  patient = np.nan_to_num(np.nanmean(patient, axis=0), nan=0)
  print(patient)
  print(patient.shape)

  test_patients_imputed[idx_patient] = patient

In [ ]:
test_patients_imputed.shape

In [ ]:
predicted_test_patients = model.predict(test_patients_imputed)

In [ ]:
predicted_test_patients.shape

In [ ]:
TESTS = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total',
         'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
         'LABEL_Bilirubin_direct', 'LABEL_EtCO2','LABEL_Sepsis']
i=0

In [ ]:
for pid in IdF_test_unique:
    output.loc[i,'pid']=int(pid)
    output.loc[i,TESTS]=predicted_test_patients[i,:]
    i+=1

In [ ]:
output

# Task C

In [ ]:
data_patients

In [ ]:
reg_patients_imputed = np.zeros((data_patients.shape[0], 35))

In [ ]:
reg_patients_imputed.shape

In [ ]:
reg_patients_imputed = np.nan_to_num(data_patients, nan=0)

In [ ]:
reg_patients_imputed = reg_patients_imputed[:,2:].reshape((-1, 12*35))

In [ ]:
reg_patients_imputed.shape

In [ ]:
reg_labels = labels.values

In [ ]:
reg_labels

In [ ]:
reg_labels = reg_labels[:,-4:]

In [ ]:
reg_labels

In [ ]:
reg_labels.shape

In [ ]:
def model_reg():
  model_reg = Sequential()
  model_reg.add(Dense(1024, kernel_initializer='he_uniform', activation='relu'))
  model_reg.add(BatchNormalization())
  model_reg.add(Dropout(0.5)) 
  model_reg.add(Dense(512, kernel_initializer='he_uniform', activation='relu'))
  model_reg.add(BatchNormalization())
  model_reg.add(Dropout(0.3))
  model_reg.add(Dense(reg_labels.shape[1]))
  model_reg.compile(loss='mean_squared_error', optimizer='adam')
  return model_reg

In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [ ]:
estimator = KerasRegressor(build_fn=model_reg, epochs=100, batch_size=128)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, reg_patients_imputed, reg_labels, cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [ ]:
model_reg = model_reg()

In [ ]:
#create callback
filepath = 'regression.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=True,
                             mode='min')

In [ ]:
from keras.callbacks import ReduceLROnPlateau

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=4, min_lr=0.00001)

In [ ]:
callbacks = [checkpoint, reduce_lr]

In [ ]:
reg_patients_imputed[100]

In [ ]:
model_reg.fit(reg_patients_imputed, reg_labels, epochs=30,validation_split=0.2,
                  callbacks=callbacks)

In [ ]:
from keras.models import load_model

In [ ]:
filepath

In [ ]:
model_reg = load_model(filepath)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
predicted_reg = model_reg.predict(reg_patients_imputed[15000:])

In [ ]:
r2_score(reg_labels[15000:], predicted_reg)

In [ ]:
test_reg_patient = test_data_patient.reshape((-1, 12*35))

In [ ]:
test_reg_patient = np.nan_to_num(test_reg_patient, nan=0)

In [ ]:
test_reg_patient

In [ ]:
predicted_test_reg = model_reg.predict(test_reg_patient)

In [ ]:
predicted_test_reg

In [ ]:
i=0
for pid in IdF_test_unique:
    output.loc[i,'pid']=int(pid)
    output.loc[i,['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']]=predicted_test_reg[i, :]
    i+=1

In [ ]:
output[:-1]

In [ ]:
compression_opts = dict(method='zip', archive_name='prediction.csv')
output[:-1].to_csv('prediction.zip', index=False, float_format='%.3f', compression=compression_opts)